# DDR3 Data DQ groups

Data pairs need to be matched separately in DQ-groups, and have more stringent requirements than the other pins:

- DQ group 0..7 bits  should be length matched together with their strobes
- DQ group 8..15 bits should be length matched together with their strobes

## Time Skew
- Table 58: DQS, DQS# to DQ skew, per access - tDQSQ: 200 ps max (DDR3L-800) (total time budget)
- Period is 2 / 800e6 = 2.5 ns
- Between clock pairs
	- 5 ps
	- (300e6 m/s * 50e-12 s) / ((sqrt(4.1) + 1) / 2) = 9.9 mm (6.635 mm in )

- Between DQS+ and DQS-: +-2ps
	- 
- Between DQ-signals: 10 ps
	- 

# DDR3 Address pins / Control pins
Address pins have less stringent requirements, but should still be matched with the CK+, CK- pairs and should (preferably) use the same bank.

# DDR3 Control signals
Should be length-matched together with its RAM_CS pins.